In [ ]:
import pandas as pd
import altair as alt
import seaborn as sns
import json

In [ ]:
df_pro = pd.read_csv("../data/ANID/BDH_Proyectos.csv",delimiter=";",decimal=",")
df_postu = pd.read_csv("../data/ANID/POSTULACIONES_2016_2020.csv",encoding='latin-1',delimiter=";",decimal=",")

In [ ]:
df_pro.columns

In [ ]:
area_filter = ["CIENCIAS MEDICAS Y DE LA SALUD"]


In [ ]:
df_pro_area_2020 = df_pro.loc[df_pro["AREA_OCDE"].isin(area_filter)&(df_pro["AÑO_CONCURSO"]==2020)]

In [ ]:
df_pro_area_2020.head()

In [ ]:
df_pro_area_2020_desc = df_pro_area_2020.groupby(by=["PROGRAMA_CONICYT","INSTRUMENTO"]).describe()
df_pro_area_2020_desc

In [ ]:
df_pro_area_2020_f = df_pro_area_2020.loc[
    ~(df_pro_area_2020["MACROZONA MINCIENCIA (segUn ax)"].isin(["SIN INFORMACION","MULTIREGIONAL"]))].copy()

In [ ]:
df_pro_area_2020_f.loc[:,"MONTO ADJUDICADO (MILLONES $)"] = (df_pro_area_2020_f["MONTO ADJUDICADO (MILES $)"]/1000)

In [ ]:
df_pro_area_2020_f["INSTRUMENTO"].unique()

In [ ]:
bar_all = alt.Chart(df_pro_area_2020_f).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    y = alt.Y('MONTO ADJUDICADO (MILLONES $):Q',aggregate="max",title="MONTO (MILLONES $)",
              axis={"titleFontSize":16,"labelFontSize":16}),
    x=alt.X('INSTRUMENTO:O',title="INSTRUMENTO",axis={"labelAngle":-90, "titleFontSize":16,"labelFontSize":16}),
    color='PROGRAMA_CONICYT:N',
    #column='Nombre Región:N'
    
).properties(
    width=720,
    height=720,
    title="Instrumentos Públicos I+D y Montos Máximos adjudicados (2020)"
).configure_title(
    fontSize=16,
    #font='Courier',
    anchor='middle',
    color='black'
)

#bar_all.save("instrumentos_montos_2020.png",scale_factor=2.0)
bar_all

In [ ]:
## PLOT por regiones
bars = alt.Chart(df_pro_area_2020_f).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    y = alt.Y('MONTO ADJUDICADO (MILLONES $):Q',aggregate="max",title="MONTO (MILLONES $)"),
    x=alt.X('INSTRUMENTO:O',title="INSTRUMENTO",axis={"labelAngle":-90}),
    color='PROGRAMA_CONICYT:N',
    #column='Nombre Región:N'
    
).properties(
    width=200,
    height=200
).facet(
    alt.Facet('MACROZONA MINCIENCIA (segUn ax):O',title=""), columns=3, center=True
).resolve_scale(
  y='independent'
)

bars



### 6 . Cuáles son las empresas privadas que hacen I+D en el área de la salud

In [ ]:
df_innova = pd.read_excel("data/portafolio_innova.xlsx")
df_innova_2020 = df_innova.loc[df_innova["anio_adjudicacion"].isin([2020])].copy()


In [ ]:
macrozona_centrosur = {"Región del Biobío":"CENTRO SUR", "Región del Libertador General Bernardo O'Higgins":"CENTRO SUR",
                       "Región del Maule":"CENTRO SUR", "Región de Ñuble":"CENTRO SUR",
                       "Región Metropolitana de Santiago": "RM"} 

df_innova_2020.loc[:,"MACROZONA"] = df_innova_2020['regionejecucion'].map(macrozona_centrosur)

In [ ]:
df_innova_2020_salud = df_innova_2020.loc[df_innova_2020["mercado_objetivo_final"]=="Salud y Farmacéutica (en humanos)"]

In [ ]:
df_innova_2020_salud.loc[df_innova_2020_salud["MACROZONA"]=="CENTRO SUR"]["beneficiario"]

In [ ]:
df_innova_2020_salud.iloc[:,:]

In [ ]:
df_innova_2020_salud_f1 = df_innova_2020_salud.dropna(subset=["MACROZONA"],axis=0)
df_innova_2020_salud_f1.loc[:,"aprobado_corfo"] = (df_innova_2020_salud_f1.loc[:,"aprobado_corfo"]/1000000).copy()

In [ ]:
df_innova_2020_salud_f1.loc[df_innova_2020_salud_f1["MACROZONA"]=="CENTRO SUR"].iloc[:,10:]

In [ ]:
## PLOT por regiones
bars = alt.Chart(df_innova_2020_salud_f1).mark_bar(
    cornerRadiusTopLeft=3,
    cornerRadiusTopRight=3
).encode(
    y = alt.Y('aprobado_corfo:Q',aggregate="max",title="MONTO (MILLONES $)"),
    x=alt.X('beneficiario:O',title="Empresas",axis={"labelAngle":-90}),
    color='instrumento:N',
    #column='Nombre Región:N'

).facet(
    alt.Facet('MACROZONA:O',title=None),
    columns=2, center=True
).resolve_scale(
    y='independent',
    x='independent'
).properties(
    title="Empresas que utilizan instrumentos I+D y Montos adjudicados (2020)"
).configure_title(fontSize=16,anchor="middle")

#bars.save("empresas_ID_2020.png",scale_factor=2.0)
bars
